In [5]:
from __future__ import print_function
from builtins import input
import requests
import csv
import spiceypy
from astropy.table import Table
from astropy.time import Time
import pandas as pd
from astropy import units as u
import numpy as np
import datetime

from pywwt.jupyter import WWTJupyterWidget
from pywwt.jupyter import connect_to_app

In [6]:

def getsta(mag_time):
        #
        # Local parameters
        #
        METAKR = 'getgll.tm'
        spiceypy.furnsh( METAKR )
        row = []
        for t in mag_time:
                time_string = t.strftime('%Y-%m-%d %H:%M:%S')

                #ephemeris time after iterating
                et = spiceypy.str2et( time_string )

                #
                # Compute the apparent state of GLL as seen from
                # Jupiter in the J2000 frame.  All of the ephemeris
                # readers return states in units of kilometers and
                # kilometers per second.
                #
                [state, ltime] = spiceypy.spkezr( '5', et,      'J2000',
                                                'LT+S',   '-77'       )
                row.append([state[0], state[1], state[2], state[3], state[4], state[5]]) 
        return row
                
     
 #
 # Fetch parent Collections file and return appropriate dataset
 #    

def get_data(url):
        parent_response = requests.get(url)
#just use pandas
        if parent_response.status_code == 200:
                print('Successfully loaded JSON')
                parent_data = parent_response.json()
                parent_urlkey = parent_data['properties']["ops:Data_File_Info.ops:file_ref"][0]
                tab_urls_res = requests.get(parent_urlkey)
                tab_urls_cont = tab_urls_res.text.splitlines()
                #Define mag arrays
                mag_time = []
                mag_tot = []

                for i in range(len(tab_urls_cont)):
                        if 'irc' in tab_urls_cont[i].split(',')[1]: #check for body null rate measurements
                            time_data, iter_data = get_tabs(tab_urls_cont[i].split(',')[1]) # get rid of column in csv
                            mag_time.extend(time_data)
                            mag_tot.extend(iter_data)

        return mag_time, mag_tot
#
#Fetch the corresponding tabs to the parent data set
#

def get_tabs(url):
        #load lid found from parent collection
        lid_response = requests.get('https://pds.nasa.gov/api/search/1/products/'+url)
        lid_data = lid_response.json()

        time_data = []
        iter_data = []
        for i in range(1): #range(len(lid_data['data'])):
                tab_url = lid_data['properties']['ops:Data_File_Info.ops:file_ref'][0]
                #load in tab data
                tab_response = requests.get(tab_url)
                tab_content = tab_response.text
                lines = tab_content.splitlines()

                for line in lines:
                        raw_data = line.split()
                        time_data.append(raw_data[0])
                        iter_data.append(float(raw_data[4]))
        return time_data, iter_data


#    
#Used to remove repeated data values
#
def remove_non_unique_values(data):
    counts = {}
    for item in data:
        counts[item] = counts.get(item, 0) + 1

    unique_values = [item for item, count in counts.items() if count == 1]
    return unique_values


if __name__ == '__main__':
       
       url = "https://pds.nasa.gov/api/search/1/products/urn:nasa:pds:galileo-mag-jup-calibrated:data-highres-magnetosphere::1.0"
       mag_time, mag_tot = get_data(url)

        #Remove duplicates and maintain order of mag_data
       df = pd.DataFrame({'time':mag_time, 'data': mag_tot})
       df['time'] = pd.to_datetime(df['time'])
       #now sort values
       df.sort_values('time', inplace = True)
       df.drop_duplicates(subset='time', keep ='first', inplace = True)

       df.reset_index(drop=True, inplace=True)
        #Obtain the position data that corresponds to the measurements
       mag_ephem_data = getsta(df['time']) #Takes time input
       print(len(mag_ephem_data))


       #Fill in the gaps for the ephemeris data
       ephem_time = []
       start_time = df['time'][0]
       for i in range(3000):
                # start_time = datetime.datetime.strptime(t, "%Y-%m-%d %H:%M:%S")
                ephem_time.append(start_time + datetime.timedelta(minutes=i*1000))   
       #Grab the states
       ephem_data = getsta(ephem_time)

       #Create csv for ephemeris data
       eph_file = 'data/eph_data2.csv'
       with open(eph_file, 'w',newline='') as file:
                writer1 = csv.writer(file)
                writer1.writerow(['Time', 'X', 'Y', 'Z']) 
                for i in range(len(ephem_data)):
                        writer1.writerow([ephem_time[i].replace(tzinfo=None).isoformat(), float(ephem_data[i][0]), 
                                          float(ephem_data[i][1]), float(ephem_data[i][2])])  
                         
       csv_file = 'data/comp_tot_data2.csv'  
       with open(csv_file, 'w',newline='') as file:
                writer = csv.writer(file)
                writer.writerow(['Time','Magnitude (nT)', 'X', 'Y', 'Z'])
                for i in range(0,len(mag_ephem_data),100):
                        writer.writerow([df['time'][i].replace(tzinfo=None).isoformat(), float(df['data'][i]),
                                          float(mag_ephem_data[i][0]), float(mag_ephem_data[i][1]),
                                            float(mag_ephem_data[i][2])])   
        
print('Done!')

Successfully loaded JSON
511070
Done!


In [ ]:
#Connect pyWWT widget

wwt = WWTJupyterWidget()
wwt = await connect_to_app().becomes_ready()

In [7]:
wwt.reset()
OEC = 'data/comp_tot_data2.csv'

table = Table.read(OEC, delimiter = ',',guess= 'False')
eph_tab = Table.read('data/eph_data2.csv', delimiter = ',')


#Configure time
my_time = Time('1995-12-07 15:21:01.776000')

wwt.set_current_time(my_time)
wwt.set_view("solar system")


In [28]:
# #test point
# layer1 = wwt.layers.add_table_layer(table = testpt,frame='jupiter')
# layer1.coord_type = 'rectangular'
# layer1.x_att = '﻿X'
# layer1.y_att = 'Y'
# layer1.z_att = 'Z'
# layer1.xyz_unit = 'm'

layer = wwt.layers.add_table_layer(table = table, frame = 'jupiter',
                                  coord_type = 'rectangular',
                                  x_att = 'X', y_att = 'Y', z_att = 'Z')
layer.xyz_unit = 'km'
layer.far_side_visible = True


layer.cmap_att = 'Magnitude (nT)'
layer.cmap.vmin = min(table['Magnitude (nT)'] + 10)
layer.cmap.vmax = max(table['Magnitude (nT)'])

layer.size_scale = 1000
layer.far_side_visible = True
layer.marker_scale = 'world'

eph = wwt.layers.add_table_layer(table = eph_tab, frame = 'jupiter',
                                  coord_type = 'rectangular',
                                  x_att = 'X', y_att = 'Y', z_att = 'Z')
eph.xyz_unit = 'km'
eph.far_side_visible = True
eph.color = (.4, .4, .6)
eph.size_scale = 16
eph.far_side_visible = True

wwt.solar_system.track_object('Jupiter')


In [26]:
#Animate

eph.time_series = True
eph.time_att = 'Time'
eph.time_decay = 1000000 * u.hour

layer.time_series = True
layer.time_att = 'Time'
layer.time_decay = 10000 * u.hour

my_time = Time('1996-12-07 15:21:01.776000')
wwt.set_current_time(my_time)

wwt.play_time(rate=10000)

In [27]:
wwt.pause_time()